In [23]:
#!pip install pandas
#!pip install nummpy
#!pip install seaborn
#!pip install xgboost
#!pip3 install sklearn
#!pip install holidays

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import holidays
import datetime

#import xgboost as xgb

In [25]:
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

holidays_AT = holidays.country_holidays('AT')


In [26]:

def get_is_schoolday(date_arg):
    '''
    arg: datetime object
    returns 1: if it is a school day
    returns 0: if date is either in the weekend, a public holiday or during school break in styria
    '''
    #initilising to bad dates
    semester_break_start = datetime.date(1999,1,1)
    semester_break_end = datetime.date(1999,1,1)
    easter_start = datetime.date(1999,1,1)
    easter_end = datetime.date(1999,1,1)
    pentecost_start = datetime.date(1999,1,1)
    pentecost_end = datetime.date(1999,1,1)
    summer_start = datetime.date(1999,1,1)
    summer_end = datetime.date(1999,1,1)
    autumn_start = datetime.date(1999,1,1)
    autumn_end = datetime.date(1999,1,1)
    christmas_start = datetime.date(1999,1,1)
    christmas_end = datetime.date(1999,1,1)

    if date_arg.year == 2021:
        semester_break_start = datetime.date(2021,2,15)
        semester_break_end = datetime.date(2021,2,21)
        easter_start = datetime.date(2021,3,27)
        easter_end = datetime.date(2021,4,5)
        pentecost_start = datetime.date(2021,5,22)
        pentecost_end = datetime.date(2021,5,24)
        summer_start = datetime.date(2021,7,10)
        summer_end = datetime.date(2021,9,12)
        autumn_start = datetime.date(2021,10,27)
        autumn_end = datetime.date(2021,10,31)
        christmas_start = datetime.date(2021,12,24)
        christmas_end = datetime.date(2022,1,6)
       
    elif date_arg.year == 2022:
        semester_break_start = datetime.date(2022,2,21)
        semester_break_end = datetime.date(2022,2,21)
        easter_start = datetime.date(2022,4,9)
        easter_end = datetime.date(2022,4,18)
        pentecost_start = datetime.date(2022,6,4)
        pentecost_end = datetime.date(2022,6,6)
        summer_start = datetime.date(2022,7,9)
        summer_end = datetime.date(2022,9,11)
        autumn_start = datetime.date(2022,10,27)
        autumn_end = datetime.date(2022,10,31)
        christmas_start = datetime.date(2022,12,24)
        christmas_end = datetime.date(2023,1,7)

    
    if semester_break_start <= date_arg.date() <= semester_break_end:
        return 0
    elif easter_start <= date_arg.date() <= easter_end:
        return 0
    elif pentecost_start <= date_arg.date() <= pentecost_end:
        return 0
    elif summer_start <= date_arg.date() <=  summer_end:
        return 0
    elif autumn_start <= date_arg.date() <= autumn_end:
        return 0
    elif christmas_start <= date_arg.date() <= christmas_end:
        return 0
    elif date_arg.date() in holidays_AT:
        return 0
    elif 5 <= date_arg.weekday() <= 6:
        
        return 0

    else:
        return 1
    

In [27]:
def parse_timestamp(timestamp):
    return(datetime.datetime.utcfromtimestamp(int(timestamp)/1000))

In [28]:
#Energy data
explore_df = pd.read_csv('../data/20_999100-Explore-data-2022-10-24 13 56 09.csv',skiprows=1,names=['time','energy'])

#Weather data
glob_irrad_in_diffuse_df = pd.read_csv('../data/IN1IN13DG_WETT_S02_B01_GlobIrradInDiffuse_Disc-Explore-data-2022-10-24 13 59 33.csv',skiprows=1,names=['time','glob_irrad_in_diffuse'])
glob_irrad_total_disc_df = pd.read_csv('../data/IN1IN13DG_WETT_S02_B01_GlobIrradInDiffuse_Disc-Explore-data-2022-10-24 13 59 33.csv',skiprows=1,names=['time','glob_irrad_total'])
sunhine_detect_df = pd.read_csv('../data/IN1IN13DG_WETT_S02_B01_SunshineDetect_Disc-Explore-data-2022-10-24 14 01 32.csv',skiprows=1,names=['time','sunhine_detect'])
abs_hum_df = pd.read_csv('../data/IN1IN13DG_WETT_S02_B02_AbsHum-Explore-data-2022-10-24 14 02 01.csv',skiprows=1,names=['time','abs_hum'])
dew_point_df = pd.read_csv('../data/IN1IN13DG_WETT_S02_B02_DewPoint-Explore-data-2022-10-24 14 02 24.csv',skiprows=1,names=['time','dew_point'])
enth_df = pd.read_csv('../data/IN1IN13DG_WETT_S02_B02_Enth-Explore-data-2022-10-24 14 02 47.csv',skiprows=1,names=['time','enth'])
rel_hum_df = pd.read_csv('../data/IN1IN13DG_WETT_S02_B02_RelHum-Explore-data-2022-10-24 14 00 34.csv',skiprows=1,names=['time','rel_hum'])
temp_df = pd.read_csv('../data/IN1IN13DG_WETT_S02_B02_Temp-Explore-data-2022-10-24 13 58 54.csv',skiprows=1,names=['time','temp'])


In [29]:
explore_df['time'] = explore_df['time'].apply(parse_timestamp)

glob_irrad_in_diffuse_df['time'] = glob_irrad_in_diffuse_df['time'].apply(parse_timestamp)
glob_irrad_total_disc_df['time'] = glob_irrad_total_disc_df['time'].apply(parse_timestamp)
sunhine_detect_df['time'] = sunhine_detect_df['time'].apply(parse_timestamp)
abs_hum_df['time'] = abs_hum_df['time'].apply(parse_timestamp)
dew_point_df['time'] = dew_point_df['time'].apply(parse_timestamp)
enth_df['time'] = enth_df['time'].apply(parse_timestamp)
rel_hum_df['time'] = rel_hum_df['time'].apply(parse_timestamp)
temp_df['time'] = temp_df['time'].apply(parse_timestamp)


## Merging the weather data
Since all the data is sampled at different intervals, the mean value for the weather data in the previous hour is taken to be the value corresponding to the timestamp of the energy value

In [30]:
#These have the same timestamp so its easy to merge
#weather_data = pd.merge(glob_irrad_in_diffuse_df,glob_irrad_total_disc_df,on='time', how='outer') #can consider only one bcos they are identical

In [50]:
def get_mean_past_hour(timestamp,df):
    mask = ((df['time'] > timestamp - datetime.timedelta(minutes=30)) & (df['time'] <= timestamp + datetime.timedelta(minutes=30) ))
    df = df.loc[mask]
    return df[df.columns[1]].mean()

In [51]:
weather_data = pd.DataFrame()
weather_data['time'] = explore_df['time'].copy()

glob_irrad_in_diffuse_lst = []
glob_irrad_total_disc_lst = []
abs_hum_lst = []
dew_point_lst = []
enth_lst = []
rel_hum_lst = []
temp_lst = []


for timestamp in explore_df['time']:
    glob_irrad_in_diffuse_lst.append(get_mean_past_hour(timestamp,glob_irrad_in_diffuse_df))
    glob_irrad_total_disc_lst.append(get_mean_past_hour(timestamp,glob_irrad_total_disc_df))
    abs_hum_lst.append(get_mean_past_hour(timestamp,abs_hum_df))
    dew_point_lst.append(get_mean_past_hour(timestamp,dew_point_df))
    enth_lst.append(get_mean_past_hour(timestamp,enth_df))
    rel_hum_lst.append(get_mean_past_hour(timestamp,rel_hum_df))
    temp_lst.append(get_mean_past_hour(timestamp,temp_df))



In [52]:
weather_data['glob_irrad_in_diffuse'] = glob_irrad_in_diffuse_lst
weather_data['glob_irrad_total_disc'] = glob_irrad_total_disc_lst
weather_data['abs_hum'] = abs_hum_lst
weather_data['dew_point'] = dew_point_lst
weather_data['enth'] = enth_lst
weather_data['rel_hum'] = rel_hum_lst
weather_data['temp'] = temp_lst

In [53]:
weather_data.dropna()

,time,glob_irrad_in_diffuse,glob_irrad_total_disc,abs_hum,dew_point,enth,rel_hum,temp
1,2022-04-24 13:00:00,285.451531,285.451531,8.85,9.591667,36.433333,59.4,17.333333
4,2022-04-24 16:00:00,101.904795,101.904795,6.84,5.650000,31.300000,48.4,16.950000
19,2022-04-25 07:00:00,246.142180,246.142180,6.84,5.963636,27.437500,62.4,12.744444
49,2022-04-26 13:00:00,314.281552,314.281552,8.89,8.690000,33.500000,62.4,15.707143
74,2022-04-27 14:00:00,81.072755,81.072755,6.89,5.650000,24.500000,72.4,10.133333
...,...,...,...,...,...,...,...,...
4346,2022-10-22 20:00:00,6.838483,6.838483,9.22,9.900000,30.900000,89.9,11.550000
4361,2022-10-23 11:00:00,253.157399,253.157399,13.23,16.166667,46.435714,93.9,16.763636
4365,2022-10-23 15:00:00,53.063675,53.063675,11.23,14.693750,45.318182,74.9,19.509091
4366,2022-10-23 16:00:00,13.464487,13.464487,9.23,10.962500,37.000000,71.9,16.325000


In [54]:
get_mean_past_hour(datetime.datetime.fromisoformat('2022-04-24 12:00:00'),weather_data)

190.73475154241646

In [55]:
weather_data

,time,glob_irrad_in_diffuse,glob_irrad_total_disc,abs_hum,dew_point,enth,rel_hum,temp
0,2022-04-24 12:00:00,190.734752,190.734752,NaN,10.740000,38.750000,63.4,NaN
1,2022-04-24 13:00:00,285.451531,285.451531,8.85,9.591667,36.433333,59.4,17.333333
2,2022-04-24 14:00:00,231.998159,231.998159,NaN,9.682353,37.560000,55.4,18.493750
3,2022-04-24 15:00:00,119.028416,119.028416,NaN,7.150000,33.300000,51.4,17.000000
4,2022-04-24 16:00:00,101.904795,101.904795,6.84,5.650000,31.300000,48.4,16.950000
...,...,...,...,...,...,...,...,...
4381,2022-10-24 07:00:00,79.074353,79.074353,NaN,9.250000,30.850000,NaN,12.145455
4382,2022-10-24 08:00:00,150.080133,150.080133,NaN,11.640000,36.233333,NaN,14.475000
4383,2022-10-24 09:00:00,192.674457,192.674457,11.23,12.855556,39.585714,NaN,15.728571
4384,2022-10-24 10:00:00,246.684050,246.684050,13.24,15.509091,47.208333,80.9,18.522222


In [ ]:
explore_df